- Description of Notebook: This analysis compares the number of orthologs between banana and human using OMA, OrthoInspector, and best-bidirectional hit.
- Last updated: 8 Dec 2020
- By: Natasha Glover

# Setup

In [1]:
#import libraries
from omadb import Client
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

#seaborn options
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'}, font_scale=1.5)
sns.set_style('whitegrid')

# Human-Banana orthologs using OrthoInspector

*OrthoInspector is a software suite for inference of orthologous relationships between protein coding-genes and an online resource to access and query precomputed orthology databases.* https://lbgi.fr/orthoinspectorv3/

In [2]:
human_ncbi_taxid = str(9606)
banana_ncbi_taxid = str(214687)

In [3]:
#get all the banana genes with an ortholog in human
response = requests.get("http://lbgi.fr/orthoinspectorv3/api/Eukaryota/species/"+ banana_ncbi_taxid +\
                        "/orthologs/"+ human_ncbi_taxid)

#count the number of banana genes with a human ortholog
banana_genes_with_orthologs = response.json()

print("There are {} banana genes with a human ortholog.".format(len(banana_genes_with_orthologs.keys())))

#count the number of banana genes
banana_genome = requests.get("http://lbgi.fr/orthoinspectorv3/api/Eukaryota/species/214687/proteins").json()

print("There are {} total number of genes in banana.".format(len(banana_genome)))

#percentage of banana genes with a human ortholog
percentage = len(banana_genes_with_orthologs.keys())/len(banana_genome)*100
print("Using OrthoInspector, {:.0f}% of the banana genome's proteins have an ortholog in human.".format(percentage))


There are 7668 banana genes with a human ortholog.
There are 36440 total number of genes in banana.
Using OrthoInspector, 21% of the banana genome's proteins have an ortholog in human.


Same thing with human.

In [4]:
#get all the human genes with an ortholog in banana
response = requests.get("http://lbgi.fr/orthoinspectorv3/api/Eukaryota/species/"+ human_ncbi_taxid +\
                        "/orthologs/"+ banana_ncbi_taxid)

#count the number of human  genes with a banana ortholog
human_genes_with_orthologs = response.json()

print("There are {} human genes with a banana ortholog.".format(len(human_genes_with_orthologs.keys())))

#count the number of human genes
human_genome = requests.get("http://lbgi.fr/orthoinspectorv3/api/Eukaryota/species/"+ human_ncbi_taxid +"/proteins").json()

print("There are {} total number of genes in human.".format(len(human_genome)))

#percentage of human genes with a banana ortholog
percentage = len(human_genes_with_orthologs.keys())/len(human_genome)*100
print("Using OrthoInspector, {:.0f}% of the human genome's proteins have an ortholog in banana.".format(percentage))


There are 5058 human genes with a banana ortholog.
There are 21006 total number of genes in human.
Using OrthoInspector, 24% of the human genome's proteins have an ortholog in banana.


# Human-Banana orthologs using OMA

Here, I use PyOMADB, the user-friendly Python wrapper around the OMA REST API. You can install it with "pip install omadb" from the command line.

In [5]:
c = Client()

First, get the number of genes in the human and banana genomes:

In [6]:
#count number of genes in the human genome
human_genome = c.genomes.proteins("HUMAN").as_dataframe()
human_genome = human_genome[human_genome['is_main_isoform']==True]

print("There are {} total number of genes in human.".format(len(human_genome)))

There are 20159 total number of genes in human.


In [7]:
#count number of genes in the banana genome
banana_genome = c.genomes.proteins("MUSAM").as_dataframe()
banana_genome = banana_genome[banana_genome['is_main_isoform']==True]

print("There are {} total number of genes in banana.".format(len(banana_genome)))

There are 36439 total number of genes in banana.


Next, get the orthologs!

In [ ]:
#get the pairwise orthologs between human and banana
human_banana_pairs = list(c.pairwise('HUMAN', 'MUSAM'))

Finally, compute the percentage of genes with an ortholog in each species:

In [ ]:
#count the number and percent of human genes with an ortholog in banana

#take the set to get a unique list, as a given gene might have multiple orthologs
human_genes_with_orthologs = set([x['entry_1']['omaid'] for x in human_banana_pairs if x['entry_1']['is_main_isoform']==True])

human_percentage = len(human_genes_with_orthologs)/len(human_genome)* 100

print("Using OMA, {:.0f}% of the human genome's proteins have an ortholog in banana.".format(human_percentage), "\n")

In [ ]:
#count the number and percent of banana genes with an ortholog in human

banana_genes_with_orthologs = set([x['entry_2']['omaid'] for x in human_banana_pairs if x['entry_2']['is_main_isoform']==True])

banana_percentage = len(banana_genes_with_orthologs)/len(banana_genome)* 100

print("Using OMA, {:.0f}% of the banana genome's proteins have an ortholog in human.".format(banana_percentage))

# Human-banana orthologs using Best Bidirectional Hit

Update:
- Downloaded human genome from UniProt: ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000005640_9606.fasta.gz
- This reference proteome has 20609 genes, one representative protein per locus.

- Downloaded the banana genome from UniProt: ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000012960_214687.fasta.gz
- 36474 genes (1 protein sequence per gene)

Commands for running blast:
- source /dcsrsoft/spack/bin/setup_dcsrsoft
- module load gcc
- module load blast-plus/2.9.0
- makeblastdb -in human_genome.faa -dbtype prot
- makeblastdb -in banana_genome.faa -dbtype prot
- mkdir banana_input
- mkdir human_input
- pyfasta split -n 100 human_genome.faa
- mv human_genome.[0-9]*.faa human_input/
- pyfasta split -n 100 banana_genome.faa
- mv banana_genome.[0-9]*.faa banana_input/

The next part uses the SLURM script.
- sbatch blast_banana_vs_human.sh
- sbatch blast_human_vs_banana.sh

- cat banana_vs_human_output/* > banana_vs_human.blastp
- cat human_vs_banana_output/* > human_vs_banana.blastp


In [ ]:
header = ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","qlen", \
          "sstart","send","slen","evalue","bitscore"]
human_df = pd.read_csv("human_vs_banana.blastp", sep="\t", header=None)
human_df.columns = header
human_df = human_df.drop_duplicates(subset="qseqid", keep="first")
human_df.rename({"qseqid":"human_gene", "sseqid":"banana_gene"}, axis=1, inplace=True)
print(len(human_df))
      
human_df[:3]


In [ ]:
banana_df = pd.read_csv("banana_vs_human.blastp", sep="\t", header=None)
banana_df.columns = header
banana_df = banana_df.drop_duplicates(subset="qseqid", keep="first")
banana_df.rename({"qseqid":"banana_gene", "sseqid":"human_gene"}, axis=1, inplace=True)
print(len(banana_df))
banana_df[:3]

In [ ]:
bbh_df = pd.merge(left=human_df, right=banana_df, on=['human_gene', 'banana_gene'], how="inner")
bbh_df[:3]

In [ ]:
nb_bbhs = len(bbh_df)
nb_human_genes_uniprot = 20609
nb_banana_genes_uniprot = 36474
print("There are {} genes in the uniprot reference human proteome."\
      .format(nb_human_genes_uniprot))
print("There are {} genes in the uniprot reference banana proteome."\
      .format(nb_banana_genes_uniprot))
print("There are {} best bidirectional hits between human and banana."\
      .format(nb_bbhs))
print("This is {:.0f}% of the human genome, and {:.0f}% of the banana genome."\
      .format(nb_bbhs/nb_human_genes_uniprot*100, nb_bbhs/nb_banana_genes_uniprot*100))

# Summary

In [ ]:
summary_df = pd.DataFrame(data={"method": ["OMA","OrthoInspector","BBH","Internet memes"], "human": [17,24,17,50], \
                                "banana":[14,21,10,50]})
df = pd.melt(summary_df, id_vars="method", value_vars=["human", "banana"])
df.rename({"variable":"species", "value": "% of genome shared with the other"}, axis=1, inplace=True)

sns.barplot(x="method", y="% of genome shared with the other", data=df, hue="species")

In [ ]:
plt.figure(figsize=(10,6))
myplot = sns.barplot(x="method", y="human", data=summary_df)
for p in myplot.patches:
    myplot.annotate(format(p.get_height(), '.0f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   size = 16,
                   textcoords = 'offset points')
plt.title("Percentage of Human Genes Orthologous to Banana", size=26)
plt.ylabel("%")
plt.savefig('./Percentage_human_banana_orthologs_barplot.pdf')

# Bonus! GO enrichment

So what are the functions of the human-banana orthologs? We can find out by doing a Gene Ontology (GO) enrichment analysis. 

To do this, I will get the canonical ID of each human gene with an ortholog and use this as the "study set" for a GOEA using the [panther tool](http://pantherdb.org/webservices/go/overrep.jsp).  

In [ ]:
#write file for upload to go enrichment tool
canonical_ids = human_genome[human_genome['omaid'].isin(human_genes_with_orthologs)]['canonicalid']

with open("human_orthologs_for_go_enrichment.txt", "w") as outfile:
    for humanid in canonical_ids:
        outfile.write(humanid + "\n")

Use the list of genes in the previous step to upload to the online go enrichment webserver. The following code works on the resulting table file, which I named "GOanalysis.txt."

In [ ]:
go_df = pd.read_csv("./GOanalysis.txt", sep="\t", skiprows=11)

#do some manipulation to get the go term and the p-value of enriched terms
go_df['go_term'] = go_df['GO biological process complete'].apply(lambda x: x.split("(")[-1])
go_df['go_term'] = go_df['go_term'].apply(lambda x: x.split(")")[0])
go_df = go_df[go_df['go_term'].str.contains("GO")]

#print for revigo
#print(go_df[['go_term','human_orthologs_for_go_enrichment.txt (FDR)']].to_string(index=False, justify="left"))

#write to file
go_df[['go_term', 'human_orthologs_for_go_enrichment.txt (FDR)']].to_csv("./enrichedGOterms_bonferroni.tsv", sep="\t", \
                                                                         index=False, header=False)

Summarize and visualize the resulting list of enriched GO terms with GO-Figure! https://www.biorxiv.org/content/10.1101/2020.12.02.408534v1.full.pdf

In [ ]:
%%bash
python ~/Applications/GO-Figure/gofigure.py -i ~/DessimozRepos/blogpost-code/enrichedGOterms.tsv \
-o gofigure/ --legend full --title "Enriched GO terms for human-banana orthologs" --cluster_labels description-numbered